In [1]:
using Flux, Statistics
using Flux: crossentropy, throttle, logitcrossentropy
using Base.Iterators: repeated, partition, flatten
using DelimitedFiles
using LinearAlgebra
using RDatasets
using GLM
using BSON: @save, @load
using Random
using Base.Threads 

In [2]:
function getEdepGateBarA0(number, angle)
    folder = "/home/jupyter-nrmeskell/dosimeter/gate_data/bars/"
    angle = lpad(angle,3,"0")
    fileName = filter(x->occursin(string("A", angle), x), readdir(folder))[number]
    filePath = string(folder, fileName)
    return Int.(round.(reshape(readdlm(filePath)[4:67,2], (8,8))))
end

getEdepGateBarA0 (generic function with 1 method)

In [3]:
#getEdepGateBarA0(99, 20)

In [4]:
function getEdepGateBarA90(number, angle)
    folder = "/home/jupyter-nrmeskell/dosimeter/gate_data/bars/"
    angle80 = lpad(angle+80, 3, "0")
    angle100 = lpad(angle+100, 3, "0")
    fileName80 = filter(x->occursin(string("A", angle80), x), readdir(folder))[number]
    fileName100 = filter(x->occursin(string("A", angle100), x), readdir(folder))[number]
    filePath80 = string(folder, fileName80)
    filePath100 = string(folder, fileName100)
    return Int.(round.(reshape(((readdlm(filePath80)[4:67,2].+readdlm(filePath100)[4:67,2])/2), (8,8))))
end

getEdepGateBarA90 (generic function with 1 method)

In [5]:
#getEdepGateBarA90(4, 260)

In [6]:
function getEdepGateGrid(number, angle)
    folder = "/home/jupyter-nrmeskell/dosimeter/gate_data/cubes/"
    angle = lpad(angle, 3, "0")
    fileName = filter(x->occursin(string("A", angle), x), readdir(folder))[number]
    filePath = string(folder, fileName)
    return Int.(round.(readdlm(filePath)[4:515,2]))
end

getEdepGateGrid (generic function with 1 method)

In [7]:
#getEdepGateGrid(1, 40)

dosimeter/gate_data/bars/W05__H05__A000_bars_extracted.txt

In [8]:
function getEdepOhioBarA0(number)
    folder = "/home/jupyter-nrmeskell/dosimeter/ohio_data/extracted_data/"
    fileName = filter(x->occursin(r"R...txt", x), readdir(folder))[number]
    filePath = string(folder,fileName)
    return Int.(round.(reshape(readdlm(filePath)[4:67,2], (8,8))))
end

getEdepOhioBarA0 (generic function with 1 method)

In [9]:
#getEdepOhioBarA0(1)

In [10]:
function getEdepOhioBarA90(number)
    folder = "/home/jupyter-nrmeskell/dosimeter/ohio_data/extracted_data/"
    fileName = filter(x->occursin(r"R.._r.txt", x), readdir(folder))[number]
    filePath = string(folder,fileName)
    return Int.(round.(reshape(readdlm(filePath)[4:67,2], (8,8))))
end

getEdepOhioBarA90 (generic function with 1 method)

In [11]:
#getEdepOhioBarA90(1)

In [12]:
function normalize(inputArray)
    min = minimum(inputArray)
    inputArray = inputArray.-min
    max = maximum(inputArray)/2048
    inputArray = inputArray./max
    return Int.(round.(inputArray))
end

normalize (generic function with 1 method)

In [13]:
#normalize(getEdepOhioBarA90(1))

In [14]:
#normalize(getEdepOhioBarA0(1))

In [15]:
function makeTrainingData()
    folder = "/home/jupyter-nrmeskell/dosimeter/gate_data/bars/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8, 8, 2, trainingSize*13)
    for i = 1:trainingSize
        for r = 1:13
            trainingArray[:, :, 1, (i-1)*14 + r] = getEdepGateBarA0(i, (r-1)*20)
            trainingArray[:, :, 2, (i-1)*14 + r] = getEdepGateBarA90(i, (r-1)*20)
        end
    end
    return trainingArray
end

makeTrainingData (generic function with 1 method)

In [16]:
function makeValidationData()
    folder = "/home/jupyter-nrmeskell/dosimeter/gate_data/bars/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8, 8, 2, trainingSize)
    for i = 1:trainingSize
        trainingArray[:, :, 1, i] = getEdepGateBarA0(i, (13)*20)
        trainingArray[:, :, 2, i] = getEdepGateBarA90(i, (13)*20)
    end
    return trainingArray
end

makeValidationData (generic function with 1 method)

In [17]:
#makeTrainingData()

In [18]:
function makeTargetData()
    folder = "/home/jupyter-nrmeskell/dosimeter/gate_data/cubes/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8*8*8, trainingSize*13)
    for i = 1:trainingSize
        for r = 1:13
            trainingArray[:,(i-1)*14 + r] = getEdepGateGrid(i, (r-1)*20)
        end
    end
    return trainingArray
end

makeTargetData (generic function with 1 method)

In [19]:
function makeValTargetData()
    folder = "/home/jupyter-nrmeskell/dosimeter/gate_data/cubes/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8*8*8, trainingSize)
    for i = 1:trainingSize
        trainingArray[:,i] = getEdepGateGrid(i, (14-1)*20)
    end
    return trainingArray
end

makeValTargetData (generic function with 1 method)

In [20]:
#makeTargetData()

In [21]:
function makeInputData()
    folder = "/home/jupyter-nrmeskell/dosimeter/ohio_data/extracted_data/"
    trainingSize = size(filter(x->occursin("_r.txt", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8, 8, 2, trainingSize)
    for i = 1:trainingSize
        trainingArray[:,:,1,i] = getEdepOhioBarA0(i)
        trainingArray[:,:,2,i] = getEdepOhioBarA90(i)
    end
    return trainingArray
end

makeInputData (generic function with 1 method)

In [22]:
#makeInputData()

In [23]:
trainingArray = makeTrainingData()

LoadError: BoundsError: attempt to access 8×8×2×1300 Array{Int64, 4} at index [1:8, 1:8, 1, 1301]

In [24]:
targetArray = makeTargetData()

LoadError: BoundsError: attempt to access 512×1300 Matrix{Int64} at index [1:512, 1301]

In [25]:
validationArray = makeValidationData()

8×8×2×100 Array{Int64, 4}:
[:, :, 1, 1] =
   403   1388   2124   2507   2734   2315   2048   1709
  1615   5549   7391   7764   6950   5730   4258   3143
 22602  45117  39016  28061  20732  13695   9221   5912
 13789  56701  94384  89460  63780  37756  22713  13545
  1390   9327  36010  68049  81088  73066  51832  30405
   658   2220   8701  19231  31318  41140  46671  42233
   322    981   2542   5890  10700  15613  20328  21931
   247    396   1005   2208   4152   6095   8184   9379

[:, :, 2, 1] =
 117  331  1889  28757  59796  9857  1188  332
 129  331  1985  33970  65509  7660   979  298
 164  280  1640  36004  61134  4896   708  192
 121  272  1623  40392  50008  2674   474  145
  89  186  1062  40141  34581  1508   364  151
  67  167   858  34295  19211   706   265   86
  52  156   569  21549   8839   491   145   77
  54  100   298   8341   2973   265   118   46

[:, :, 1, 2] =
   595   1610   2600   3037   3179   2523   2000   1767
  5440   9810  11295   9432   8353   6687   47

In [26]:
validationTargetArray = makeValTargetData()

512×100 Matrix{Int64}:
   6    8   13    6    7    7    6  …    16    17    13    10    16     9
  13    9   15   10   12   14    8       30    14    15    41    19    11
  22    8   14    7    6    8    7       39    12    15    54    20    35
  36   12   15    7    9   14    9       54    20    28    16     8    31
   9   13   10   14   15    8    7       23    38    18    12    21    30
   9   16   41    6   12   11   12  …    15     7    19    38    21    19
  17    9    4    6   13   10    4        8    11     9    15    18    13
  13    5    5    3    6    8    5       12    26    16    28    37    11
  20   27   16   12   18   21   13      553   635   553   416   312   350
  27   41   49   18   17   15   25      729   644   502   856   526   263
  27   28   27   35   23   24   34  …  1044   690   736   659   640   340
  23   20   27   39   27   22   37      828   474   746   592   387   331
  26   21   57   46   33   46   41      739   695   628   703   479   331
   ⋮           

In [27]:
inputArray = makeInputData()

8×8×2×21 Array{Int64, 4}:
[:, :, 1, 1] =
 38484  42128  52850  126117  208366  54216  42413  38391
 39382  44555  47446  118228  285328  61556  41870  39128
 36595  39956  55851  189571  294911  57231  42411  38908
 38021  43502  45917  122588  483994  71271  40778  39157
 40115  46611  55757  176140  345909  74524  42913  39929
 39228  38664  58439  162217  244550  68536  43572  39139
 37466  37933  56898  131986  361086  60613  48624  37953
 35604  38155  43116   84692  184161  70698  43053  37902

[:, :, 2, 1] =
 19404  22059  44432  101296   77978  36257  21430  19332
 20006  24228  42568   87020  105097  53825  21343  20054
 17905  20826  56275  160868  105372  47073  21694  19664
 19379  23974  36643   91733  144345  65212  20635  20134
 20977  26756  56546  140444  126791  68834  22822  20629
 20002  20012  61299  128934   87739  61529  22612  20076
 18627  19083  62967   99105  119800  47310  25929  19225
 17006  19348  31036   58054   65617  59364  22150  18946

[:, :, 1, 2] =

In [28]:
m = Chain(Conv((2,2), 2=>16, pad=(1,1), relu),
        x -> maxpool(x, (2,2)),

        Conv((2,2), 16=>32, pad=(1,1), relu),
        x -> maxpool(x, (1,1)),
        
        x -> reshape(x, :, size(x,4)), 
        Dense(800, 8*8*4),
        Dense(8*8*4, 8*8*8))

Chain(Conv((2, 2), 2=>16, relu), #23, Conv((2, 2), 16=>32, relu), #24, #25, Dense(800, 256), Dense(256, 512))

In [29]:
loss(x, y) = mean((m(x)-y).^2)
opt = ADAM()
dataset = (repeated((trainingArray, targetArray), 100))
evalcb = () -> @show (loss(trainingArray, targetArray)) 
CurrentR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(trainingArray)[:,1]), B=targetArray[:,1])))
print("Training R2: ", CurrentR2, '\n')
CurrentVR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(validationArray)[:,1]), B=validationTargetArray[:,1])))
print("Validation R2: ", CurrentR2, '\n')

LoadError: UndefVarError: trainingArray not defined

In [30]:
for i = 1:10
    Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 2))
    CurrentR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(trainingArray)[:,1]), B=targetArray[:,1])))
    print("Training R2: ", CurrentR2, '\n')
    CurrentVR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(validationArray)[:,1]), B=validationTargetArray[:,1])))
    print("Validation R2: ", CurrentR2, '\n')
end

LoadError: UndefVarError: evalcb not defined